In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

from utilidades.validar import validar
from utilidades.graficar import ver_dispersion

In [15]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

mejorPruebaActual = pd.read_csv('submits/limpieza_descripciones_retuneo_LGBM_300d.csv')

In [3]:
#trainMejorado = pd.read_csv("data/train_limpio4_bis.csv")
#testMejorado = pd.read_csv("data/test_limpio4.csv")

In [4]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 71 columns):
tipodepropiedad_Apartamento                      240000 non-null int64
tipodepropiedad_Casa en condominio               240000 non-null int64
tipodepropiedad_Casa                             240000 non-null int64
tipodepropiedad_Terreno                          240000 non-null int64
tipodepropiedad_Terreno comercial                240000 non-null int64
tipodepropiedad_Local Comercial                  240000 non-null int64
tipodepropiedad_Quinta Vacacional                240000 non-null int64
tipodepropiedad_Oficina comercial                240000 non-null int64
tipodepropiedad_Edificio                         240000 non-null int64
tipodepropiedad_Casa uso de suelo                240000 non-null int64
tipodepropiedad_Local en centro comercial        240000 non-null int64
tipodepropiedad_Bodega comercial                 240000 non-null int64
tipodepropiedad_Otros                

In [5]:
mejorPruebaActual.head()

,id,target
0,4941,5.400875e+06
1,51775,1.025289e+06
2,115253,2.250194e+06
3,299321,1.244130e+06
4,173570,6.153558e+05


In [6]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

# Se define la funcion

In [11]:
def generar_subida(DFTRAIN, metodo, DFTEST, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)    
    
    X, y = DFTRAIN.drop( columns = ['precio'] ), DFTRAIN["precio"]

    metodo.fit(X,y)
    
    subida = pd.DataFrame( metodo.predict(DFTEST) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    #Este modelo no necesita las euristicas,pero igual no da muy bien
    
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

#### Evalúo los parametros iniciales

In [8]:
GradientBoostingRegressor()

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [16]:
#dfinicial1 = dfinicial[dfinicial['metrostotales']<225].drop( columns = ['dia'] )
#dftest1 = dftest.drop( columns = ['dia'] )


#dfinicial1  = trainMejorado[trainMejorado['metrostotales']<225].drop( columns = ['dia'] )
#dftest1 = testMejorado.drop( columns = ['dia'] )


#dfinicial1 = dfinicial2.drop( columns = [ 'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos',  'escuelascercanas'
#                           ])                                        
 
#dftest1 = dftest2.drop( columns = [  'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos', 'escuelascercanas'
#                           ])   

reg = GradientBoostingRegressor(
)



X, y, subida = generar_subida( dfinicial, reg, dftest, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO 
#(No es la mejor forma pero sirve como aproximacion si el Kfold tarda mucho)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

540277.0

### Verifico formalmente el metodo con Kfold

In [18]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor, False)

Error de fold N°0 es : 796954.7472719415
Error de fold N°1 es : 797055.6763865778
Error de fold N°2 es : 798668.9923507419
Error de fold N°3 es : 796861.2225620802
Error de fold N°4 es : 790163.0381804889


# Modifico los hiperparametros propios del Gradient Boosting para maximizar su eficacia

In [22]:

gradientBoostingOptimizado = GradientBoostingRegressor(
    alpha=0.9, #no cambia nada
    max_depth=4,
    n_estimators=100, #el default es 100 asique deberia dar bastante
    learning_rate=0.5,
    min_samples_leaf=250,
    subsample=0.5,
    validation_fraction=0.1
)

    
X, y, subida = generar_subida( dfinicial, gradientBoostingOptimizado, dftest, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )    

488512.0

In [21]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 715277.06581662
Error de fold N°1 es : 705703.5965765929
Error de fold N°2 es : 709066.5240343931
Error de fold N°3 es : 706036.579450608
Error de fold N°4 es : 704949.2174846162


# Quito las propiedades muy grandes porque perjudican el calculo

In [23]:
dfinicial1 = dfinicial[dfinicial['metrostotales']<225]#.drop( columns = ['dia'] )
dftest1 = dftest#.drop( columns = ['dia'] )
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )    

488765.0

In [24]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 588995.4266792418
Error de fold N°1 es : 586227.4738019543
Error de fold N°2 es : 591839.4391255038
Error de fold N°3 es : 585896.726326693
Error de fold N°4 es : 589426.8528888001


In [25]:
#aclarar que el primero subio y el segundo bajo porque el primero 
#igual calcula las propiedades grandes y el segundo no,pero igual esto
#prueba que las propiedades grandes enturbiecen el resultado

In [26]:
dfinicial1 = dfinicial.drop( columns = ['dia'] )
dftest1 = dftest.drop( columns = ['dia'] )
    
X, y, subida = generar_subida( dfinicial1, gradientBoostingOptimizado, dftest1, "prueba_andres2.csv", False)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(mejorPruebaActual, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )  

484402.0

In [27]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor,alpha=0.9,max_depth=4,n_estimators=100,learning_rate=0.5,min_samples_leaf=250,subsample=0.5,validation_fraction=0.1)

Error de fold N°0 es : 714109.9543325376
Error de fold N°1 es : 704573.8540342319
Error de fold N°2 es : 713927.6567804025
Error de fold N°3 es : 713185.0902547204
Error de fold N°4 es : 704212.5961690441


In [28]:
#sacar el dia no afectó nada,probablemente el 
#mismo algoritmo al determinar que no aporta 
#informacion decide obviarlo cuando está,por 
#lo que no cambia nada si está o no

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(y_test, pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel("Valores reales", fontsize=16)
ax.set_ylabel("Valores predichos", fontsize=16);

In [ ]:
#reg = LinearRegression(normalize=True, n_jobs = -1)
#reg.fit(X,y)

In [ ]:
train.info()

In [ ]:
train['descripcion'] = train['descripcion'].to_string()
train.info()

In [ ]:
train.head()